
#  TPS-11-2021: Catboost
## Table of Contents
* [1. Overview](#1.)
* [2. Setup](#2.)
* [3. Exploratory Data Analysis & Preprocessing](#3.)
	* [3.1 Statistic Info](#3.1)
	* [3.2 Correlation Score](#3.2)
	* [3.3 Distribution of Label](#3.3)
	* [3.4 Drop ID column](#3.4)
	* [3.5 Train Validation Split](#3.5)
* [4. Model Development](#4.)
* [5. Submission](#5.)

<a id="1."></a>
## 1. Overview
In this Notebook, I will build a Tabular Prediciton Model using Catboost.
<a id="2."></a>
## 2. Setup

In [ ]:
import numpy as np
import pandas as pd
try:
  from catboost import CatBoostClassifier
except:
  !pip install catboost
  from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
import os
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
class Config:
    is_kaggle_platform = os.path.exists("/kaggle/input")
    dataset_name = "tabular-playground-series-nov-2021"
    data_path = "/kaggle/input/%s/"%(dataset_name) if is_kaggle_platform else ""
    submit_filename = "submission.csv"
    label_name = "target"
config = Config()

In [ ]:
if not config.is_kaggle_platform:
  try:
    import kaggle
  except:
    !pip install kaggle
  if not os.path.exists("/root/.kaggle/kaggle.json"):
    !echo "{"username":"{your username}","key":"{your apikey}"}" >> /root/.kaggle/kaggle.json
    !chmod 600 /root/.kaggle/kaggle.json
  !kaggle competitions download -c $config.dataset_name
  !unzip test.csv.zip
  !unzip train.csv.zip
  !unzip sample_submission.csv.zip

In [ ]:
train = pd.read_csv(config.data_path + "train.csv")
test = pd.read_csv(config.data_path + "test.csv")
sample_submission = pd.read_csv(config.data_path + "sample_submission.csv")


<a id="3."></a>
## 3. Exploratory Data Analysis & Preprocessing

In [ ]:
train.head()

<a id="3.1"></a>
### 3.1 Statistic Info

In [ ]:
train.info()

In [ ]:
train.describe()

<a id="3.2"></a>
### 3.2 Correlation Score

In [ ]:
corr = train.corr()
corr

In [ ]:
corr.sort_values(ascending=False, inplace=True, by=config.label_name, key= lambda x: abs(x))
corr[config.label_name]

In [ ]:
correlated_columns = corr[config.label_name][corr[config.label_name].abs() > 0.05].index
correlated_columns, len(correlated_columns)

In [ ]:
correlation_score = train.corr()
correlated_features = correlation_score[config.label_name].sort_values(ascending=False).dropna()
correlated_columns = list(correlated_features[correlated_features.abs() > 0.05].index)
correlated_columns.remove(config.label_name)
print(correlated_columns)

In [ ]:
corr2 = train[correlated_columns].corr()
corr2

In [ ]:
plt.figure(figsize=(20, 20))
sns.heatmap(corr2, annot=True)

<a id="3.3"></a>
### 3.3 Distribution of Label

In [ ]:
sns.countplot(x=config.label_name, data=train)

<a id="3.4"></a>
### 3.4 Drop ID column
ID is not needed. So remove this column.

In [ ]:
train.pop("id")
_ = test.pop("id")

<a id="3.5"></a>
### 3.5 Train Validation Split

In [ ]:
train_features, val_features = train_test_split(train, test_size=0.15, random_state=42)
train_targets = train_features.pop(config.label_name)
val_targets = val_features.pop(config.label_name)
train_features.head()

There isn't any missing value, not need for Data Wrangling

In [ ]:
null_counts = train_features.isnull().sum()
print(null_counts[null_counts > 0])
null_counts = val_features.isnull().sum()
print(null_counts[null_counts > 0])

<a id="4."></a>
## 4. Model Development

In [ ]:

cat_params = {
    'iterations': 15000,
    'objective': 'CrossEntropy',
    'bootstrap_type': 'Bernoulli',
    'od_wait': 1000,
    'learning_rate': 0.023575206684596582,
    'reg_lambda': 36.30433203563295,
    'random_strength': 43.75597655616195,
    'depth': 7,
    'min_data_in_leaf': 11,
    'leaf_estimation_iterations': 1,
    'subsample': 0.8227911142845009,
    'task_type' : 'GPU',
    'devices' : '0',
    'verbose' : 1000,
    'eval_metric': 'AUC'
}
cat = CatBoostClassifier(**cat_params)
cat.fit(train_features, train_targets, eval_set=(val_features, val_targets))


<a id="5."></a>
## 5. Submission

In [ ]:
y_pred = cat.predict_proba(test)
sample_submission[config.label_name] = y_pred[:, 1]
sample_submission.to_csv(config.submit_filename, index=False)
if not config.is_kaggle_platform:
  !kaggle competitions submit $config.dataset_name -m "Submission" -f $config.submit_filename

**If you found my notebook useful, give me an upvote.**

You may have a look at some of my TPS notebooks before.

- [Tabular Playground Series Prediction(Aug 2021)](https://www.kaggle.com/lonnieqin/tabular-playground-series-prediction)
- [Tabular Playground Prediction(Sep 2021) with CatBoost](https://www.kaggle.com/lonnieqin/catboost-tabular-playground-prediction-sep-2021)
- [Tabular Prediction(Oct 2021) with CatBoost](https://www.kaggle.com/lonnieqin/catboost-tabular-prediction-oct-2021)
- [TPS Prediction with DNN and KerasTuner (Oct 2021)](https://www.kaggle.com/lonnieqin/tps-prediction-with-dnn-and-kerastuner-oct-2021)
- [TPS-10-21: DNN](https://www.kaggle.com/lonnieqin/tps-10-21-dnn)
